In [1]:
#SETUP always run this
import math
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json

sectors = json.load(open("sectors.json",'r'))
closingPrices = pd.read_csv("latestClose.csv")
closingPrices = closingPrices.drop(columns=["Date"])
closingPrices.tail()

allSectors = list(sectors.keys())
allIndustries = []
for sector in allSectors:
    allIndustries.extend(list(sectors[sector].keys()))
allIndustries = list(set(allIndustries))
allTickers = []
for sector in allSectors:
    for industry in sectors[sector]:
        allTickers.extend(sectors[sector][industry])
allTickers = list(set(allTickers))

industries = {}
for sector in allSectors:
    industries[sector] = list(sectors[sector].keys())

print("sectors:", allSectors)
print(len(industries))
print(len(allTickers))

sectors: ['Consumer Discretionary', 'Technology', 'Health Care', 'Financial', 'Real Estate', 'Communication Services', 'Consumer Staples', 'Industrial', 'Energy', 'Materials', 'Utilities']
11
3390


In [ ]:
def getAllData(tickers):
    anchorData = yf.download(tickers, start = "2021-01-01")
    anchorData.head()
    closingPrices = anchorData["Close"]
    closingPrices.to_csv("latestClose.csv")
    closingPrices = pd.read_csv("latestClose.csv")
    return closingPrices

def getSearchRange(start, end):
    return closingPrices[closingPrices['Date'].between(start, end)]

closingPrices = getAllData(tickers)


In [2]:
industries

{'Consumer Discretionary': ['Durable Household Products',
  'Auto Parts',
  'Business Training Agencies',
  'Specialty Retailers',
  'Restaurants & Bars',
  'Broadline Retailers',
  'Footwear',
  'Special Consumer Services',
  'Home Construction',
  'Apparel Retailers',
  'Recreational Products',
  'Hotels',
  'Furnishings',
  'Gambling',
  'Clothing & Accessories',
  'Toys',
  'Home Improvement Retailers',
  'Automobiles',
  'Recreational Services',
  'Tires',
  'Travel & Tourism'],
 'Technology': ['Software',
  'Semiconductors',
  'Electronic Equipment',
  'Telecom Equipment',
  'Electrical Components',
  'Computer Services',
  'Computer Hardware',
  'Renewable Energy Equipment'],
 'Health Care': ['Pharmaceuticals',
  'Biotechnology',
  'Health Care Providers',
  'Medical Supplies',
  'Medical Equipment'],
 'Financial': ['Banks',
  'Property-Casualty Insurance',
  'Consumer Finance',
  'Financial Administration',
  'Asset Managers',
  'Investment Services',
  'Full Line Insurance',
 

In [3]:
movingAverages = {}
for sector in allSectors:
    print(sector)
    for industry in sectors[sector]:
        count = 0
        total = 0
        for ticker in sectors[sector][industry]:
            movingAverages[(ticker, 20)] = closingPrices[ticker][-20::].mean()
            movingAverages[(ticker, 50)] = closingPrices[ticker][-50::].mean()
            movingAverages[(ticker, 200)] = closingPrices[ticker][-200::].mean()

Consumer Discretionary
Technology
Health Care
Financial
Real Estate
Communication Services
Consumer Staples
Industrial
Energy
Materials
Utilities


In [ ]:
print(len(movingAverages), movingAverages)

In [9]:
MAs = [20,50,200]
counts = {}
for sector in allSectors:
    sectorCount = {}
    sectorTotal = 0
    print(sector)
    for industry in sectors[sector]:
        count = {}
        total = 0
        for ticker in sectors[sector][industry]:
            sectorTotal +=1
            total +=1
            for ma in MAs:
                if closingPrices[ticker].iloc[-1] > movingAverages.get((ticker, ma),0):
                    count[ma] = count.get(ma,0)+1
                    sectorCount[ma] = sectorCount.get(ma,0)+1

        for i in count:
            print(industry, i, count[i], total)

        percents = {i : count[i]/total for i in count}
        counts[industry] = percents
    sectorPercent = {i : sectorCount[i]/sectorTotal for i in sectorCount}
    counts[sector] = sectorPercent
    for i in sectorCount:
        print(sector, i, sectorCount[i], sectorTotal)
    

Consumer Discretionary
Durable Household Products 50 2 13
Durable Household Products 200 3 13
Durable Household Products 20 3 13
Auto Parts 200 8 22
Auto Parts 20 1 22
Business Training Agencies 200 4 14
Business Training Agencies 50 1 14
Specialty Retailers 20 11 53
Specialty Retailers 200 23 53
Specialty Retailers 50 9 53
Restaurants & Bars 200 24 34
Restaurants & Bars 50 10 34
Restaurants & Bars 20 11 34
Broadline Retailers 20 6 13
Broadline Retailers 50 3 13
Broadline Retailers 200 4 13
Footwear 50 2 7
Footwear 200 5 7
Footwear 20 3 7
Special Consumer Services 200 16 36
Special Consumer Services 20 6 36
Special Consumer Services 50 4 36
Home Construction 20 16 20
Home Construction 200 19 20
Home Construction 50 18 20
Apparel Retailers 200 8 25
Apparel Retailers 20 1 25
Recreational Products 20 4 25
Recreational Products 50 4 25
Recreational Products 200 9 25
Hotels 50 2 16
Hotels 200 9 16
Furnishings 200 3 13
Furnishings 20 1 13
Furnishings 50 1 13
Gambling 20 1 20
Gambling 50 4 20

In [10]:
percents = {}
for sector in allSectors:
    percents[sector] = {}
    percents[sector]["Sector"] = counts[sector]
    for industry in sectors[sector]:
        percents[sector][industry] = counts[industry]
percents

{'Consumer Discretionary': {'Sector': {50: 0.20971867007672634,
   200: 0.47058823529411764,
   20: 0.20971867007672634},
  'Durable Household Products': {50: 0.15384615384615385,
   200: 0.23076923076923078,
   20: 0.23076923076923078},
  'Auto Parts': {200: 0.36363636363636365, 20: 0.045454545454545456},
  'Business Training Agencies': {200: 0.2857142857142857,
   50: 0.07142857142857142},
  'Specialty Retailers': {20: 0.20754716981132076,
   200: 0.4339622641509434,
   50: 0.16981132075471697},
  'Restaurants & Bars': {200: 0.7058823529411765,
   50: 0.29411764705882354,
   20: 0.3235294117647059},
  'Broadline Retailers': {20: 0.46153846153846156,
   50: 0.23076923076923078,
   200: 0.3076923076923077},
  'Footwear': {50: 0.2857142857142857,
   200: 0.7142857142857143,
   20: 0.42857142857142855},
  'Special Consumer Services': {200: 0.4444444444444444,
   20: 0.16666666666666666,
   50: 0.1111111111111111},
  'Home Construction': {20: 0.8, 200: 0.95, 50: 0.9},
  'Apparel Retailers

In [11]:
#Print Percents to a CSV. Organized by Sector, then Industry
with open("percentages.csv", "w") as f:
    f.write("Moving Average Breadth Dashboard,20,50,200,\n")
    f.write("\n")
    for sector in allSectors:
        f.write(sector.upper() + ",")
        for ma in MAs:
            output = round(percents[sector]["Sector"][ma]*100,2)
            f.write(str(output) + ",")
        f.write("\n")
        for industry in sectors[sector]:
            f.write(industry + ",")
            for ma in MAs:
                if ma in percents[sector][industry]:
                    output = round(percents[sector][industry][ma]*100,2)
                    f.write(str(output) + ",")
                else:
                    f.write("0,")
            f.write("\n")
        f.write("\n")
        
    

In [12]:
percents["Consumer Discretionary"]

{'Sector': {50: 0.20971867007672634,
  200: 0.47058823529411764,
  20: 0.20971867007672634},
 'Durable Household Products': {50: 0.15384615384615385,
  200: 0.23076923076923078,
  20: 0.23076923076923078},
 'Auto Parts': {200: 0.36363636363636365, 20: 0.045454545454545456},
 'Business Training Agencies': {200: 0.2857142857142857,
  50: 0.07142857142857142},
 'Specialty Retailers': {20: 0.20754716981132076,
  200: 0.4339622641509434,
  50: 0.16981132075471697},
 'Restaurants & Bars': {200: 0.7058823529411765,
  50: 0.29411764705882354,
  20: 0.3235294117647059},
 'Broadline Retailers': {20: 0.46153846153846156,
  50: 0.23076923076923078,
  200: 0.3076923076923077},
 'Footwear': {50: 0.2857142857142857,
  200: 0.7142857142857143,
  20: 0.42857142857142855},
 'Special Consumer Services': {200: 0.4444444444444444,
  20: 0.16666666666666666,
  50: 0.1111111111111111},
 'Home Construction': {20: 0.8, 200: 0.95, 50: 0.9},
 'Apparel Retailers': {200: 0.32, 20: 0.04},
 'Recreational Products': 

In [7]:
sectors["Utilities"]["Water"]

['ARTNA', 'GWRS', 'YORW', 'MSEX', 'AWR', 'CWT', 'SBS', 'SJW', 'AWK', 'WTRG']